In [ ]:
import numpy as np

In [ ]:
class Agent:
    def __init__(self, id, num_actions, learning_method='cumulative_mean', M=None, weights=None):
        self.id = id
        self.learning_method = learning_method
        self.M = M
        self.weights = weights
        self.theta = None
        self.history = []
        self.num_actions = num_actions
        self.strategy_counts = np.zeros(num_actions)
        self.total_count = 0
        self.strategy = None

    def predict(self, x):
        if self.learning_method == 'cumulative_mean':
            return self.cumulative_mean()
        elif self.learning_method == 'last_m_mean':
            return self.last_m_mean()
        elif self.learning_method == 'weighted_moving_average':
            return self.weighted_moving_average()

    def cumulative_mean(self):
        if len(self.history) == 0:
            return 0
        return np.mean(self.history)

    def last_m_mean(self):
        if len(self.history) < self.M:
            return 0
        return np.mean(self.history[-self.M:])

    def weighted_moving_average(self):
        if len(self.history) < len(self.weights):
            return 0
        return np.dot(self.weights, self.history[-len(self.weights):])

    def update(self, y):
        self.history.append(y)
    
    def best_response(self, opponent_strategy, payoff_matrix):
        expected_payoffs = np.dot(payoff_matrix, opponent_strategy)
        best_action = np.argmax(expected_payoffs)
        return best_action

    def update_strategy(self, action):
        self.strategy_counts[action] += 1
        self.total_count += 1

    def get_strategy(self):
        if self.total_count == 0:
            return np.ones(self.num_actions) / self.num_actions
        else:
            return self.strategy_counts / self.total_count
        
    def set_strategy(self, strategy):
        self.strategy = strategy

    def play(self):
        if self.strategy is None:
            raise ValueError("Strategy not set for the agent.")
        return np.random.choice(range(len(self.strategy)), p=self.strategy)

In [ ]:
class System:
    def __init__(self, num_agents):
        self.agents = [Agent(i) for i in range(num_agents)]

    def run(self, num_periods):
        for t in range(num_periods):
            for agent in self.agents:
                x = self.get_info(agent)
                y_pred = agent.predict(x)
                action = self.take_action(agent, y_pred)
                y_true = self.get_target(agent, action)
                agent.update(y_true)

    def get_info(self, agent):
        # Получение информации об окружающем мире для агента
        pass

    def take_action(self, agent, y_pred):
        # Выбор действия агентом на основе прогноза
        pass

    def get_target(self, agent, action):
        # Получение фактического значения целевой переменной после выполнения действия
        pass


In [ ]:
class MatrixRobinsonGame:
    def __init__(self, payoff_matrix):
        self.payoff_matrix = payoff_matrix
        self.m1, self.m2 = payoff_matrix.shape
        self.player1_strategy = np.zeros(self.m1)
        self.player2_strategy = np.zeros(self.m2)

    def brown_robinson(self, num_iterations):
        for t in range(num_iterations):
            j1 = np.random.choice(self.m1, p=self.player1_strategy)
            j2 = np.random.choice(self.m2, p=self.player2_strategy)
            
            self.player1_strategy[j1] += 1
            self.player2_strategy[j2] += 1
            
            self.player1_strategy /= np.sum(self.player1_strategy)
            self.player2_strategy /= np.sum(self.player2_strategy)
        
        return self.player1_strategy, self.player2_strategy

In [ ]:
class MatrixFictitiousGame:
    def __init__(self, payoff_matrix):
        self.payoff_matrix = payoff_matrix
        self.m1, self.m2 = payoff_matrix.shape
        self.player1 = Agent(1, self.m1)
        self.player2 = Agent(2, self.m2)

    def fictitious_play(self, num_iterations):
        for t in range(num_iterations):
            player1_strategy = self.player1.get_strategy()
            player2_strategy = self.player2.get_strategy()

            player1_action = self.player1.best_response(player2_strategy, self.payoff_matrix)
            player2_action = self.player2.best_response(player1_strategy, -self.payoff_matrix.T)

            self.player1.update_strategy(player1_action)
            self.player2.update_strategy(player2_action)

        return self.player1.get_strategy(), self.player2.get_strategy()

In [ ]:
# Пример использования
payoff_matrix = np.array([[1, -1], [-1, 1]])
game = MatrixRobinsonGame(payoff_matrix)
player1_strategy, player2_strategy = game.brown_robinson(num_iterations=1000)

print("Player 1 strategy:", player1_strategy)
print("Player 2 strategy:", player2_strategy)

game = MatrixFictitiousGame(payoff_matrix)
player1_strategy, player2_strategy = game.fictitious_play(num_iterations=1000)

print("Player 1 strategy:", player1_strategy)
print("Player 2 strategy:", player2_strategy)

system = System(num_agents=10)
system.run(num_periods=100)

In [ ]:
class System:
    def __init__(self, num_agents):
        self.agents = [Agent(i) for i in range(num_agents)]

    def run(self, num_periods, game):
        for t in range(num_periods):
            for i in range(0, len(self.agents), 2):
                agent1 = self.agents[i]
                agent2 = self.agents[i+1]
                payoff1, payoff2 = game.play(agent1, agent2)
                agent1.update(payoff1)
                agent2.update(payoff2)

In [ ]:
class MatrixGame:
    def __init__(self, payoff_matrix):
        self.payoff_matrix = payoff_matrix

    def play(self, agent1, agent2):
        action1 = agent1.play()
        action2 = agent2.play()
        payoff = self.payoff_matrix[action1, action2]
        return payoff, -payoff

In [ ]:
# Пример использования
payoff_matrix = np.array([[0, -1, 1],
                          [1, 0, -1],
                          [-1, 1, 0]])
game = MatrixGame(payoff_matrix)

agent1 = Agent(0)
agent1.set_strategy([1/3, 1/3, 1/3])

agent2 = Agent(1)
agent2.set_strategy([1/3, 1/3, 1/3])

system = System(num_agents=2)
system.run(num_periods=100, game=game)